In [2]:
import sys
print(sys.executable)
print(sys.version)

!{sys.executable} -m pip show mediapipe

!python3.10 -m pip install -Uqq mediapipe

/opt/anaconda3/bin/python3.10
3.10.0 (default, Mar  3 2022, 03:54:28) [Clang 12.0.0 ]
Name: mediapipe
Version: 0.10.21
Summary: MediaPipe is the simplest way for researchers and developers to build world-class ML solutions and applications for mobile, edge, cloud and the web.
Home-page: https://github.com/google/mediapipe
Author: The MediaPipe Authors
Author-email: mediapipe@google.com
License: Apache 2.0
Location: /opt/anaconda3/lib/python3.10/site-packages
Requires: absl-py, attrs, flatbuffers, jax, jaxlib, matplotlib, numpy, opencv-contrib-python, protobuf, sentencepiece, sounddevice
Required-by: 


In [7]:
import pandas as pd
import mediapipe as mp
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
import cv2


# Paths to your datasets and images
data_dir = Path('./data')
image_labels_csv = data_dir / 'image_labels.csv'
bodyfat_csv = data_dir / 'bodyfat_dataset.csv'
images_dir = Path('images'_  # Adjust if needed

# Load CSVs
df_labels = pd.read_csv(image_labels_csv)
df_bodyfat = pd.read_csv(bodyfat_csv)

print(f"Labels df shape: {df_labels.shape}")
print(f"Bodyfat df shape: {df_bodyfat.shape}")

# Setup mediapipe pose detector
mp_pose = mp.solutions.pose

def crop_upper_body(image_path):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"Warning: Could not read {image_path}")
        return None
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(img_rgb)
        if not results.pose_landmarks:
            print(f"No pose landmarks found for {image_path}")
            return None
        
        landmarks = results.pose_landmarks.landmark
        
        # Get coordinates of shoulders and hips (indices from mediapipe pose model)
        # Left shoulder (11), right shoulder (12), left hip (23), right hip (24)
        y_coords = [landmarks[i].y for i in [11, 12, 23, 24]]
        x_coords = [landmarks[i].x for i in [11, 12]]
        
        h, w, _ = img.shape
        
        ymin = int(min(y_coords) * h)
        ymax = int(max(y_coords) * h)
        xmin = int(min(x_coords) * w)
        xmax = int(max(x_coords) * w)

        # Add some padding if desired
        pad_y = int(0.1 * (ymax - ymin))
        pad_x = int(0.1 * (xmax - xmin))

        ymin = max(ymin - pad_y, 0)
        ymax = min(ymax + pad_y, h)
        xmin = max(xmin - pad_x, 0)
        xmax = min(xmax + pad_x, w)

        cropped = img[ymin:ymax, xmin:xmax]
        cropped_rgb = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
        return Image.fromarray(cropped_rgb)

# Create folder for cropped images
cropped_dir = Path('cropped_images')
cropped_dir.mkdir(exist_ok=True)

cropped_paths = []
failed_images = []

# Loop through images, crop, and save
for idx, row in tqdm(df_labels.iterrows(), total=len(df_labels)):
    filename = row['filename'] if 'filename' in row else row.get('image_path', None)
    if not filename:
        print(f"No filename found in row {idx}")
        continue
    
    img_path = images_dir / filename
    cropped_img = crop_upper_body(img_path)
    if cropped_img is not None:
        save_path = cropped_dir / filename
        cropped_img.save(save_path)
        cropped_paths.append(save_path)
    else:
        failed_images.append(filename)

print(f"Done! Cropped {len(cropped_paths)} images, failed on {len(failed_images)} images.")

# Optionally: create a new dataframe linking cropped images with labels
df_labels['cropped_path'] = [str(p) for p in cropped_paths] + [None]*len(failed_images)
df_labels.to_csv(data_dir / 'image_labels_cropped.csv', index=False)


Labels df shape: (1596, 2)
Bodyfat df shape: (806, 10)


  0%|          | 0/1596 [00:00<?, ?it/s]

TypeError: unsupported operand type(s) for /: 'str' and 'str'